# 预测

In [4]:
import torch

# model = torch.load('../word2vec_data/nn_model/nn_model_epoch_13.pkl')
model = torch.load("../nn/nn_models/nn_model_epoch_8.pkl")
model


LSTM_attention(
  (embedding): Embedding(54848, 50)
  (encoder): LSTM(50, 100, num_layers=4, batch_first=True, dropout=0.25, bidirectional=True)
  (decoder1): Linear(in_features=200, out_features=100, bias=True)
  (decoder2): Linear(in_features=100, out_features=2, bias=True)
)

# 可视化

In [3]:
def read_word2id(file_path):
    word_to_idx = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                word, idx = line.split()
                word_to_idx[word] = int(idx)
    return word_to_idx

# 使用示例：
word_to_idx = read_word2id('../word2vec_data/word2id.txt')

In [4]:
import torch
import jieba

def classify_review(comment):
    words = list(jieba.cut(comment, cut_all=True))
    word_indices = [word_to_idx[word] if word in word_to_idx else word_to_idx['_PAD_'] for word in words]

    # 将词索引转换为张量
    words_tensor = torch.tensor(word_indices).unsqueeze(0)
    print("words_tensor: ", words_tensor)

    # 使用模型进行分类
    with torch.no_grad():
        embeddings = model.embedding(words_tensor)  # [batch, seq_len] => [batch, seq_len, embed_dim]
        states, hidden = model.encoder(embeddings)  # [batch, seq_len, embed_dim]
        output = model.decoder1(states[:, -1, :])
        output = model.decoder2(output)
        print("output:", output)
        _, pred = torch.max(output, 1)
        print("pred: ", pred)

    if pred.item() == 0:
        return '好评'
    elif pred.item() == 1:
        return '坏评'

In [5]:
# sen_model_epoch_10.pkl：真好看： 好评
# sen_model_epoch_0.pkl：真好看：差评
# 示例用法
comment = input("请输入你的评论：")
classification = classify_review(comment)
print("评论分类：", classification)


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/mc/p53lyyvn6374my8kzhqr8w840000gn/T/jieba.cache
Loading model cost 0.681 seconds.
Prefix dict has been built successfully.


words_tensor:  tensor([[    0, 16246,   536]])
output: tensor([[ 0.1968, -0.3772]])
pred:  tensor([0])
评论分类： 好评
